In [1]:
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from IPython.display import display, clear_output
pn.extension('tabulator')

import hvplot.pandas

In [2]:
sample = pd.read_csv("C:\\Users\Lloyd\Documents\GitHub\DataAnalyticsFirst\GlobalWeatherRepository.csv")
sample['last_updated'] = pd.to_datetime(sample['last_updated'])
sample[sample['country'] == "Philippines"]

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
138,Philippines,Manila,14.6,120.98,Asia/Manila,1693302300,2023-08-29 17:45:00,27.0,80.6,Light rain,...,1.2,4.1,1,1,05:44 AM,06:10 PM,04:53 PM,03:28 AM,Waxing Gibbous,93
333,Philippines,Manila,14.6,120.98,Asia/Manila,1693364400,2023-08-30 11:00:00,31.0,87.8,Partly cloudy,...,2.2,5.9,1,1,05:44 AM,06:10 PM,05:46 PM,04:34 AM,Full Moon,98
528,Philippines,Manila,14.6,120.98,Asia/Manila,1693439100,2023-08-31 07:45:00,25.0,77.0,Light rain,...,3.0,6.8,1,1,05:44 AM,06:10 PM,05:46 PM,04:34 AM,Full Moon,98
723,Philippines,Manila,14.6,120.98,Asia/Manila,1693525500,2023-09-01 07:45:00,27.0,80.6,Heavy rain at times,...,2.8,6.3,1,1,05:44 AM,06:09 PM,06:32 PM,05:39 AM,Waning Gibbous,100
918,Philippines,Manila,14.6,120.98,Asia/Manila,1693611900,2023-09-02 07:45:00,27.0,80.6,Light rain,...,4.0,8.4,1,1,05:44 AM,06:08 PM,07:17 PM,06:40 AM,Waning Gibbous,99
1112,Philippines,Manila,14.6,120.98,Asia/Manila,1693698300,2023-09-03 07:45:00,27.0,80.6,Light rain,...,4.2,8.3,1,1,05:44 AM,06:08 PM,07:59 PM,07:39 AM,Waning Gibbous,94
1307,Philippines,Manila,14.6,120.98,Asia/Manila,1693783800,2023-09-04 07:30:00,25.0,77.0,Light rain,...,3.9,7.9,1,1,05:44 AM,06:07 PM,08:41 PM,08:36 AM,Waning Gibbous,88
1502,Philippines,Manila,14.6,120.98,Asia/Manila,1693870200,2023-09-05 07:30:00,28.0,82.4,Light rain,...,5.0,10.3,1,1,05:44 AM,06:06 PM,09:25 PM,09:33 AM,Waning Gibbous,79
1697,Philippines,Manila,14.6,120.98,Asia/Manila,1693955700,2023-09-06 07:15:00,28.0,82.4,Partly cloudy,...,10.2,23.9,1,1,05:44 AM,06:05 PM,10:10 PM,10:30 AM,Waning Gibbous,70
1892,Philippines,Manila,14.6,120.98,Asia/Manila,1694041200,2023-09-07 07:00:00,27.0,80.6,Partly cloudy,...,36.0,59.6,2,4,05:45 AM,06:05 PM,10:58 PM,11:28 AM,Last Quarter,60


In [3]:
isample = sample.interactive()

# Line plot for temperature and time (JL)

In [4]:
# Slider widget
slider = pn.widgets.IntSlider(name='Data slider', start=1, end=100, step=1, value=1)
slider

IntSlider(end=100, name='Data slider', start=1, value=1)

In [5]:
datetime_range_slider = pn.widgets.DatetimeRangeSlider(
    name='Datetime Range Slider',
    start=dt.datetime(2023, 9, 5,14,0,0), end=dt.datetime(2023, 9, 30,0,0),
    value=(dt.datetime(2023, 9, 8,0,0), dt.datetime(2023, 9, 28,0,0)),
    step=10000
)

datetime_range_slider

DatetimeRangeSlider(end=datetime.datetime(2023, ..., name='Datetime Range Slider', start=datetime.datetime(2023, ..., step=10000, value=(datetime.datetime(2023, ..., value_end=datetime.datetime(2023, ..., value_start=datetime.datetime(2023, ...)

In [6]:
# Radio buttons for CO2 measures
radio = pn.widgets.RadioButtonGroup(
    name='Temperature', 
    options=['temperature_celsius','temperature_fahrenheit'],
    button_type='success'
)
countries = ['Philippines','United States of America','Canada','Germany','China','Japan','United Kingdom','Australia','India','Russia']

In [7]:
temperature_time_pipeline = (isample[(isample.country.isin(countries))]
    .groupby(['country','location_name','last_updated'])[radio].max()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

temperature_time_pipeline

In [8]:
temperature_time_lineplot = temperature_time_pipeline.hvplot(x='last_updated',by='country',y=radio,line_width=2,title="Temperature by Country")
temperature_time_lineplot

# Scatterplot and table for air quality and location (JL)

In [9]:
airquality_location_pipeline = (isample[(isample.country.isin(countries))]
    .groupby(['country','location_name','air_quality_PM10'])['air_quality_PM2.5'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

airquality_location_pipeline

In [10]:
airquality_location_pipeline2 = (isample[(isample.country.isin(countries))]
    .groupby(['country','latitude','longitude','air_quality_PM10'])['air_quality_PM2.5'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

In [11]:
airquality_location_table = airquality_location_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 15) 
airquality_location_table

In [12]:
airquality_location_table2 = airquality_location_pipeline2.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 15) 
airquality_location_table2

In [13]:
airquality_location_scatterplot = airquality_location_pipeline.hvplot(x='country',y='air_quality_PM2.5',by='location_name',
                                                                     hover_cols=['air_quality_PM10'],
                                                                    size=80,kind='scatter',
                                                                    alpha=0.5,
                                                                    legend='top',
                                                                    height=500,
                                                                    width=700,
                                                                    )

airquality_location_scatterplot

# Bar chart

In [14]:
air_index_radio = pn.widgets.RadioButtonGroup(
    name='Air quality index', 
    options=['air_quality_us-epa-index','air_quality_gb-defra-index'], 
    button_type='success'
)

In [15]:
air_index_pipeline = (isample[(isample.country.isin(countries))]
    .groupby(['country'])[air_index_radio].max()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

air_index_pipeline

In [16]:
air_index_barplot = air_index_pipeline.hvplot(kind='bar',x='country',y=air_index_radio,title='Air Quality Index')
air_index_barplot

# Pie Chart About Weather Conditions (Gab)

In [17]:
weather_column = ['country', 'condition_text']
weather = sample[weather_column]

# Define the countries to show
countries_to_show = ['Philippines', 'United States of America', 'Canada', 'Germany', 'China', 'Japan', 
                     'United Kingdom', 'Australia', 'Switzerland', 'India', 'Russia']

# Create a dropdown widget
country_dropdown = widgets.Dropdown(options=countries_to_show, description='Select a country:')
output_plot = widgets.Output()

In [18]:
select = pn.widgets.Select(name='Select', options=countries)
select

Select(options=['Philippines', ...], value='Philippines')

In [19]:
def generate_pie_chart(selected_country):
    with output_plot:
        # Clear previous output
        clear_output(wait=True)

        # Filter the DataFrame for the selected country
        filtered_data = weather[weather['country'] == selected_country]
        condition_counts = filtered_data['condition_text'].value_counts()

        # Plot a pie chart using Matplotlib
        plt.figure(figsize=(8, 5))
        plt.pie(condition_counts, labels=condition_counts.index, autopct='%1.1f%%', startangle=180)
        plt.title(f'Weather Condition Distribution in {selected_country}\n\n')
        plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        plt.show()

In [20]:
def on_country_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        generate_pie_chart(change['new'])

# Call the function to generate the initial pie chart
generate_pie_chart(countries_to_show[0])

# Call the function when the dropdown value changes
country_dropdown.observe(on_country_dropdown_change)

# Display the dropdown and the pie chart side by side in a flexbox layout
flexbox_layout = widgets.Layout(display='flex', flex_flow='column', justify_content='center', align_items='center')
display(widgets.VBox([country_dropdown, output_plot], layout=flexbox_layout))

# Create dashboard

In [21]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Global Weather Report Update', 
    sidebar=[pn.pane.Markdown("# World Weather Repository ( Daily Updating )"), 
             pn.pane.Markdown("This dataset provides daily weather information for capital cities around the world. Unlike forecast data, this dataset offers a comprehensive set of features that reflect the current weather conditions around the world."), 
             pn.pane.PNG('https://storage.googleapis.com/kaggle-datasets-images/3678699/6383099/61a1c18c08506e236bf0cd0394c09110/dataset-cover.jpg?t=2023-08-29-10-36-14', sizing_mode='scale_both'),
            pn.pane.Markdown('## Group 14'),
            pn.pane.Markdown('### Member 1'),
            pn.pane.Markdown('### Member 2'),
            pn.pane.Markdown('### Member 3')],
    main=[pn.Row(pn.Column(airquality_location_scatterplot.panel(), margin=(0,25)),
                airquality_location_table.panel(width=600)),
         pn.Row(pn.Column(temperature_time_lineplot.panel(height=500),margin=(0,25)),
               air_index_barplot.panel(height=500))],
    accent_base_color="#88d8b0",
    header_background="#444444",
)
template.show()
template.servable();

Launching server at http://localhost:64128
